In [ ]:
!pip install selenium webdriver-manager beautifulsoup4 pandas

In [ ]:
!pip list

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

def step1_collect_links():
    # --- [1. 리눅스(WSL)용 크롬 설정] ---
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # 화면 없이 실행 (필수)
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1920,1080")
    
    # 봇 탐지 회피용 헤더
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36")

    print("🚀 크롤러 시동 거는 중...")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    collected_links = [] # 수집한 링크를 저장할 리스트

    try:
        # --- [2. 검색 페이지 접속] ---
        url = "https://rollcall.com/factbase/trump/search/"
        print(f"🌐 접속 중: {url}")
        driver.get(url)
        time.sleep(5) # 페이지 로딩 대기

        # --- [3. 스크롤 다운 (데이터 더 불러오기)] ---
        # 스크롤을 많이 내릴수록 더 과거의 링크까지 가져옵니다.
        # 테스트를 위해 5번만 내립니다. (원하면 숫자를 늘리세요)
        scroll_count = 5
        for i in range(scroll_count):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            print(f"⬇️ 스크롤 다운 ({i+1}/{scroll_count})...")
            time.sleep(3) # 로딩 기다림

        # --- [4. HTML 분석 및 링크 추출] ---
        print("🔍 링크 찾는 중...")
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # [핵심] 아까 발견한 'View Transcript' 버튼만 찾기
        buttons = soup.select('a[title="View Transcript"]')

        for btn in buttons:
            href = btn['href']
            
            # 주소가 상대주소(/factbase/...)인 경우 도메인 붙여주기
            if "rollcall.com" not in href:
                full_link = "https://rollcall.com" + href
            else:
                full_link = href
            
            # 중복 아니면 리스트에 추가
            if full_link not in collected_links:
                collected_links.append(full_link)

        print("-" * 50)
        print(f"✅ 수집 성공! 총 {len(collected_links)}개의 링크를 찾았습니다.")
        print("-" * 50)
        
        # 찾은 링크 5개만 예시로 출력
        for i, link in enumerate(collected_links[:5]):
            print(f"{i+1}. {link}")

    except Exception as e:
        print(f"❌ 에러 발생: {e}")

    finally:
        driver.quit()
        print("👋 브라우저 종료")
        
    return collected_links

# --- 실행 ---
link_list = step1_collect_links()

🚀 크롤러 시동 거는 중...
🌐 접속 중: https://rollcall.com/factbase/trump/search/


In [ ]:
link_list